In [ ]:
from args import args, config
from items_dataset import items_dataset
from torch.utils.data import DataLoader
from models import Model_Crf, Model_Softmax
from transformers import AutoTokenizer
import prediction
import torch


In [ ]:
directory = "../model/"
model_name = "roberta_crf_epoch_1.pt"
device = torch.device('cuda', 0) if torch.cuda.is_available() else torch.device('cpu')
if args.use_crf:
    model = Model_Crf(config).to(device)
else:
    model = Model_Softmax(config).to(device)
model.load_state_dict(state_dict=torch.load(directory + model_name, map_location=device))

In [ ]:
input_dict = [{"span_labels":[]}]
input_dict[0]["original_text"] = "媽媽在醫院支援武漢前線醫護發回的消息，這段期間切記不要穿帶毛領或是絨線的衣服外套，較容易吸附病毒，請廣而告之。"
tokenizer = AutoTokenizer.from_pretrained(args.pre_model_name, add_prefix_space=True)
prediction_dataset = items_dataset(tokenizer, input_dict, args.label_dict)
prediction_loader = DataLoader(prediction_dataset, batch_size=args.batch_size, shuffle=True, collate_fn=prediction_dataset.collate_fn)
result = prediction.test_predict(prediction_loader, device, model)

In [ ]:
print(result[0])